In [97]:
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras import Model

## Preprocessing

In [98]:
df_decisions = pd.read_csv('./stock_decisions.csv')
df_decisions.head()

,Current Value of Stock,Expected Value of Stock,Error in expected value of stock,Percentage of portfolio,Percentage of sector,Positive,Negative,Neutral,Total,Neutral_Pos,Neutral_Neg,Sector of stock,Buy/Sell/Keep
0,0.016909,0.000000,0.057341,0.747695,0.353394,0.634112,0.393651,0.209993,0.363674,0.354623,0.288915,IT,Keep
1,0.006546,0.003842,0.060510,0.875622,0.132926,0.659470,0.618285,0.282692,0.270249,0.399351,0.499607,IT,Keep
2,0.000000,0.030124,0.046401,0.038742,0.334798,0.612328,0.595365,0.277965,0.318172,0.316199,0.429087,IT,Buy
3,0.027272,0.042233,0.108571,0.334854,0.926111,0.890981,0.371262,0.382849,0.149471,0.807705,0.553326,IT,Sell
4,0.037013,0.058251,0.039837,0.165370,0.620705,0.575857,0.650335,0.326131,0.332083,0.251870,0.433689,IT,Buy


In [99]:
df_decisions.drop('Sector of stock', axis=1, inplace=True)

In [100]:
one_hot_encoded = pd.get_dummies(df_decisions['Buy/Sell/Keep'], prefix='Buy/Sell/Keep')
one_hot_encoded = one_hot_encoded.astype(int)

In [101]:
df_decision_one_hot = df_decisions.drop(labels = 'Buy/Sell/Keep', axis=1)
df_decision_one_hot = pd.concat([df_decision_one_hot, one_hot_encoded], axis=1)

In [102]:
df_decision_one_hot.head()

,Current Value of Stock,Expected Value of Stock,Error in expected value of stock,Percentage of portfolio,Percentage of sector,Positive,Negative,Neutral,Total,Neutral_Pos,Neutral_Neg,Buy/Sell/Keep_Buy,Buy/Sell/Keep_Keep,Buy/Sell/Keep_Sell
0,0.016909,0.000000,0.057341,0.747695,0.353394,0.634112,0.393651,0.209993,0.363674,0.354623,0.288915,0,1,0
1,0.006546,0.003842,0.060510,0.875622,0.132926,0.659470,0.618285,0.282692,0.270249,0.399351,0.499607,0,1,0
2,0.000000,0.030124,0.046401,0.038742,0.334798,0.612328,0.595365,0.277965,0.318172,0.316199,0.429087,1,0,0
3,0.027272,0.042233,0.108571,0.334854,0.926111,0.890981,0.371262,0.382849,0.149471,0.807705,0.553326,0,0,1
4,0.037013,0.058251,0.039837,0.165370,0.620705,0.575857,0.650335,0.326131,0.332083,0.251870,0.433689,1,0,0


## Implementing K means clustering

In [103]:
k = 10
kmeans = KMeans(n_clusters=k)
kmeans.fit(df_decision_one_hot)

KMeans(n_clusters=10)

In [104]:
centroids = kmeans.cluster_centers_

In [105]:
centroids

array([[ 2.21862911e-01,  2.21439485e-01,  1.35696257e-01,
         5.25001630e-01,  5.04295443e-01,  3.86127107e-01,
         2.92961859e-01,  2.44283775e-01,  5.90389810e-01,
         3.74832602e-01,  1.98801460e-01,  5.55111512e-17,
         1.00000000e+00, -2.22044605e-16],
       [ 2.90120472e-01,  2.20815455e-01,  1.69018881e-01,
         5.47666399e-01,  4.59150193e-01,  3.66706979e-01,
         3.58077485e-01,  2.48617390e-01,  5.94741435e-01,
         3.61867748e-01,  2.29664455e-01,  5.55111512e-17,
        -5.55111512e-17,  1.00000000e+00],
       [ 7.22174293e-01,  8.01790987e-01,  1.63855727e-01,
         4.86943838e-01,  4.83869700e-01,  4.42923939e-01,
         3.07664943e-01,  2.64076481e-01,  5.32401992e-01,
         3.76053731e-01,  2.27943201e-01,  1.00000000e+00,
         1.66533454e-16, -1.66533454e-16],
       [ 2.57314513e-01,  3.17699521e-01,  1.50235016e-01,
         2.39466059e-01,  5.82155804e-01,  4.09631314e-01,
         2.65058593e-01,  2.42340489e-01,  5.

## Getting predictions for next day

In [106]:
from tensorflow.keras.models import load_model

In [107]:
main_df = []
main_df.append({'df': pd.read_csv('./Combining Stock and Twitter Data/MyDrive/AmazonStock_Sentiment_6M.csv').drop(['date','Date','Name'],axis=1), 'comp': 'AMZN'})
main_df.append({'df': pd.read_csv('./Combining Stock and Twitter Data/MyDrive/AAPLStock_sentiment_6M.csv').drop(['date','Date','Name'],axis=1), 'comp': 'AAPL'})
main_df.append({'df': pd.read_csv('./Combining Stock and Twitter Data/MyDrive/CSCOStock_Sentiment_6M.csv').drop(['date','Date','Name'],axis=1), 'comp': 'CSCO'})
main_df.append({'df': pd.read_csv('./Combining Stock and Twitter Data/MyDrive/IBMStock_Sentiment_6M.csv').drop(['date','Date','Name'],axis=1), 'comp': 'IBM'})
main_df.append({'df': pd.read_csv('./Combining Stock and Twitter Data/MyDrive/JNJStock_Sentiment_6M.csv').drop(['date','Date','Name'],axis=1), 'comp': 'JNJ'})
main_df.append({'df': pd.read_csv('./Combining Stock and Twitter Data/MyDrive/JNPRStock_Sentiment_6M.csv').drop(['date','Date','Name'],axis=1), 'comp': 'JNPR'})
main_df.append({'df': pd.read_csv('./Combining Stock and Twitter Data/MyDrive/MSFTStock_Sentiment_6M.csv').drop(['date','Date','Name'],axis=1), 'comp': 'MSFT'})
main_df.append({'df': pd.read_csv('./Combining Stock and Twitter Data/MyDrive/ORCLStock_Sentiment_6M.csv').drop(['date','Date','Name'],axis=1), 'comp': 'ORCL'})
main_df.append({'df': pd.read_csv('./Combining Stock and Twitter Data/MyDrive/PfizerStock_Sentiment_6M.csv').drop(['date','Date','Name'],axis=1), 'comp': 'PFIZER'})
main_df.append({'df': pd.read_csv('./Combining Stock and Twitter Data/MyDrive/TGTStock_Sentiment_6M.csv').drop(['date','Date','Name'],axis=1), 'comp': 'TGT'})

In [108]:
scalers_stock = []
for i in range(len(main_df)):
    scaler = MinMaxScaler(feature_range=(0, 1))
    main_df[i]['df'][main_df[i]['df'].columns] = scaler.fit_transform(main_df[i]['df'][main_df[i]['df'].columns])
    scalers_stock.append({'scaler': scaler, 'comp': main_df[i]['comp']})

In [109]:
from tensorflow.keras.losses import MeanSquaredLogarithmicError, MeanSquaredError
from keras.layers import Dense, LSTM, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.saving import register_keras_serializable
@register_keras_serializable(package="MyLoss")
def MyLoss(y_true, y_pred):
    msle = MeanSquaredLogarithmicError()(y_true[:,0], y_pred[:,0])
    mse = MeanSquaredError()(y_true[:,1],y_pred[:,1])
    return  msle + mse

@register_keras_serializable(package='MyModel')
class MyModel(Model):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__()
        self.lstm1 = LSTM(units=50, return_sequences=True,
                          input_shape=[None,30,13])
        self.bn1 = BatchNormalization()
        self.dropout1 = Dropout(0.2)

        self.lstm2 = LSTM(units=60, return_sequences=True)
        self.bn2 = BatchNormalization()
        self.dropout2 = Dropout(0.3)

        self.lstm3 = LSTM(units=70, return_sequences=True)
        self.bn3 = BatchNormalization()
        self.dropout3 = Dropout(0.4)

        self.lstm4 = LSTM(units=80)
        self.bn4 = BatchNormalization()
        self.dropout4 = Dropout(0.5)

        self.dense1_1 = Dense(20)
        self.dense2_1 = Dense(1)

        self.dense1_2 = Dense(20)
        self.dense2_2 = Dense(1)
        self.concat = Concatenate(axis=-1)

    def call(self, inputs):
        enc = self.lstm1(inputs)
        enc = self.bn1(enc)
        enc = self.dropout1(enc)

        enc = self.lstm2(enc)
        enc = self.bn2(enc)
        enc = self.dropout2(enc)

        enc = self.lstm3(enc)
        enc = self.bn3(enc)
        enc = self.dropout3(enc)

        enc = self.lstm4(enc)
        enc = self.bn4(enc)
        enc = self.dropout4(enc)

        out1 = self.dense1_1(enc)
        out1 = self.dense2_1(out1)

        out2 = self.dense1_2(enc)
        out2 = self.dense2_2(out2)

        out = self.concat([out1, out2])
        return out

In [110]:

def makeNextDayVector(filtered_df, todays_date, stock, comp_name):


    filtered_df['diff']=filtered_df['high']-filtered_df['low']
    filtered_df['exp']=(filtered_df['open']+filtered_df['close'])/2


    loaded_model = load_model('model.keras')

    curr_row = filtered_df[filtered_df['Date'] == todays_date]
    curr_row.drop(['date', 'Date', 'Name'], axis=1, inplace=True)
    filtered_df.drop(['date', 'Date', 'Name'], axis=1, inplace=True)
    curr_stock_value = curr_row['open'].iloc[0]

    # find scaler
    scaler = MinMaxScaler()
    for dict in scalers_stock:
        if dict['comp'] == comp_name:
            scaler = dict['scaler']
            break

    filtered_df[filtered_df.columns] = scaler.fit_transform(filtered_df[filtered_df.columns])

    predicted_values = pd.DataFrame()

    predicted_values[['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative','Neutral', 'Total', 'NeutralPos', 'NeutralNeg']] = 0

    predicted_values[['diff', 'exp']] = loaded_model.predict(np.array([filtered_df.values]).astype(float))

    # predicted_values=scaler.inverse_transform(predicted_values)
    print(predicted_values.columns)

    predicted_values[predicted_values.columns] = scaler.inverse_transform(predicted_values)



    predicted_values.drop(['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative','Neutral', 'Total', 'NeutralPos', 'NeutralNeg'], axis=1, inplace=True)


    expected_value = predicted_values['exp'].iloc[0]
    error = predicted_values['diff'].iloc[0]

    percen_portfolio = stock['Percentage of portfolio']
    percen_sector = stock['Percentage of sector']
    sector = stock['Sector of Stock']

    positives = curr_row['Positve'].iloc[0]
    negatives = curr_row['Negative'].iloc[0]
    total = curr_row['Total'].iloc[0]
    neutral = curr_row['Neutral'].iloc[0]
    neutral_pos = curr_row['NeutralPos'].iloc[0]
    neutral_neg = curr_row['NeutralNeg'].iloc[0]



    stock_vect = []
    stock_vect.append(curr_stock_value)
    stock_vect.append(expected_value)
    stock_vect.append(error)
    stock_vect.append(percen_portfolio)
    stock_vect.append(percen_sector)
    stock_vect.append(positives)
    stock_vect.append(negatives)
    stock_vect.append(total)
    stock_vect.append(neutral)
    stock_vect.append(neutral_pos)
    stock_vect.append(neutral_neg)

    # add posi negi from filtered data in stock vector

    # stock_vect.append(sector)


    return stock_vect


### User input vector

In [111]:
input = []

todays_date = '1/5/2018'
list_of_stocks = []
list_of_stocks.append({'Percentage of portfolio': 0.5, 'Percentage of sector': 15.56, 'Sector of Stock': 'IT', 'comp': 'AMZN'});
list_of_stocks.append({'Percentage of portfolio': 4.5, 'Percentage of sector': 7.8, 'Sector of Stock': 'IT', 'comp': 'MSFT'});
list_of_stocks.append({'Percentage of portfolio': 7.5, 'Percentage of sector': 18.26, 'Sector of Stock': 'Health', 'comp': 'AAPL'});
# list_of_stocks.append({'Percentage of portfolio': 16.5, 'Percentage of sector': 10.26, 'Sector of Stock': 'Health', 'comp': 'PFIZER'});
list_of_stocks

[{'Percentage of portfolio': 0.5,
  'Percentage of sector': 15.56,
  'Sector of Stock': 'IT',
  'comp': 'AMZN'},
 {'Percentage of portfolio': 4.5,
  'Percentage of sector': 7.8,
  'Sector of Stock': 'IT',
  'comp': 'MSFT'},
 {'Percentage of portfolio': 7.5,
  'Percentage of sector': 18.26,
  'Sector of Stock': 'Health',
  'comp': 'AAPL'}]

In [112]:
next_vectors = []
for i in range(len(list_of_stocks)):
    if(list_of_stocks[i]['comp'] == 'AMZN'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/AmazonStock_Sentiment_6M.csv');
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i], 'AMZN'), 'comp': 'AMZN'})
    elif(list_of_stocks[i]['comp'] == 'AAPL'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/AAPLStock_sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i], 'AAPL'), 'comp': 'AAPL'})
    elif(list_of_stocks[i]['comp'] == 'MSFT'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/MSFTStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i], 'MSFT'), 'comp': 'MSFT'})
    elif(list_of_stocks[i]['comp'] == 'PFIZER'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/PfizerStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i], 'PFIZER'), 'comp': 'PFIZER'})
    elif(list_of_stocks[i]['comp'] == 'CSCO'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/CSCOStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i], 'CSCO'), 'comp': 'CSCO'})
    elif(list_of_stocks[i]['comp'] == 'IBM'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/IBMStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i], 'IBM'), 'comp': 'IBM'})
    elif(list_of_stocks[i]['comp'] == 'JNJ'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/JNJStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i], 'JNJ'), 'comp': 'JNJ'})
    elif(list_of_stocks[i]['comp'] == 'JNPR'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/JNPRStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i], 'JNPR'), 'comp': 'JNPR'})
    elif(list_of_stocks[i]['comp'] == 'ORCL'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/ORCLStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i], 'ORCL'), 'comp': 'ORCL'})
    elif(list_of_stocks[i]['comp'] == 'TGT'):
        df = pd.read_csv('./Combining Stock and Twitter Data/MyDrive/TGTStock_Sentiment_6M.csv')
        filtered_df = df[df['Date'].replace('/', '') <= todays_date.replace('/', '')].tail(30)
        next_vectors.append({'next_day_vector': makeNextDayVector(filtered_df, todays_date, list_of_stocks[i], 'TGT'), 'comp': 'TGT'})


/home/satvikv/apps/miniconda/miniconda3/envs/rec_proj/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/tmp/ipykernel_4330/2993926521.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_row.drop(['date', 'Date', 'Name'], axis=1, inplace=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
Index(['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative',
       'Neutral', 'Total', 'NeutralPos', 'NeutralNeg', 'diff', 'exp'],
      dtype='object')


/home/satvikv/apps/miniconda/miniconda3/envs/rec_proj/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/tmp/ipykernel_4330/2993926521.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_row.drop(['date', 'Date', 'Name'], axis=1, inplace=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
Index(['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative',
       'Neutral', 'Total', 'NeutralPos', 'NeutralNeg', 'diff', 'exp'],
      dtype='object')


/home/satvikv/apps/miniconda/miniconda3/envs/rec_proj/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/tmp/ipykernel_4330/2993926521.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_row.drop(['date', 'Date', 'Name'], axis=1, inplace=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
Index(['open', 'high', 'low', 'volume', 'close', 'Positve', 'Negative',
       'Neutral', 'Total', 'NeutralPos', 'NeutralNeg', 'diff', 'exp'],
      dtype='object')


In [113]:
next_vectors

[{'next_day_vector': [1217.51,
   1395.839195690751,
   25.248018814325256,
   0.5,
   15.56,
   52.54403131,
   6.653620352,
   1022,
   40.80234834,
   52.54403131,
   27.86822061],
  'comp': 'AMZN'},
 {'next_day_vector': [87.66,
   93.4781572419405,
   1.5348345750689567,
   4.5,
   7.8,
   39.01581722,
   11.42355009,
   569,
   49.56063269,
   41.36386314,
   11.42355009],
  'comp': 'MSFT'},
 {'next_day_vector': [173.44,
   168.18254580177367,
   1.6677195708751704,
   7.5,
   18.26,
   48.41208366,
   9.295120062,
   1291,
   42.29279628,
   48.41208366,
   17.84049339],
  'comp': 'AAPL'}]

## Calculating Distance from Cluster Centroid

In [114]:
centroid_decision = {}
for i in range(len(centroids)):
	if centroids[i, -3] == max(centroids[i, -3], centroids[i, -2], centroids[i, -1]):
		centroid_decision[i] = "Buy"
	elif centroids[i, -2] == max(centroids[i, -3], centroids[i, -2], centroids[i, -1]):
		centroid_decision[i] = "Keep"
	elif centroids[i, -1] == max(centroids[i, -3], centroids[i, -2], centroids[i, -1]):
		centroid_decision[i] = "Sell"
	else:
		centroid_decision[i] = "Keep"

In [115]:
centroid_decision

{0: 'Keep',
 1: 'Sell',
 2: 'Buy',
 3: 'Buy',
 4: 'Keep',
 5: 'Sell',
 6: 'Buy',
 7: 'Buy',
 8: 'Keep',
 9: 'Sell'}

In [116]:
centroids_without_decision = centroids[:,:-3]

In [117]:
distance_from_centroid = []
for j in range(len(next_vectors)):
	current_distances = []
	for i in range(centroids_without_decision.shape[0]):
		current_distances.append(np.linalg.norm(next_vectors[j]['next_day_vector'] - centroids_without_decision[i]))
	distance_from_centroid.append(current_distances)


In [118]:
distance_from_centroid

[[2117.131669496942,
  2117.0907885773427,
  2116.4509736282675,
  2117.046468537646,
  2116.556112893096,
  2116.505127447568,
  2116.74701318854,
  2117.041197761728,
  2116.6141359432013,
  2116.624532703984],
 [587.9767144324669,
  587.9628057547883,
  587.7922214567403,
  587.954533498612,
  587.8253517868176,
  587.80863615214,
  587.7858512722622,
  587.9562731402534,
  587.8392686323014,
  587.771532566745],
 [1315.81817391519,
  1315.8056573543795,
  1315.6587062357285,
  1315.801200295975,
  1315.6920384119446,
  1315.6741065096464,
  1315.6409129878862,
  1315.8023923480573,
  1315.7003038445644,
  1315.6322094786517]]

In [119]:
results = {}

for i in range(len(distance_from_centroid)):
    min_index = 0
    for j in range(len(distance_from_centroid[i])):
        if(distance_from_centroid[i][j] < distance_from_centroid[i][min_index]):
            min_index = j

    centroid = centroid_decision[min_index]
    if centroid not in results:
        results[centroid] = []  # Initialize empty list if centroid not in results
    results[centroid].append(next_vectors[i]['comp'])


In [120]:
results

{'Buy': ['AMZN'], 'Sell': ['MSFT', 'AAPL']}